In [1]:
import os
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
import urllib.request
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, \
                            roc_auc_score, confusion_matrix, classification_report, \
                            matthews_corrcoef, cohen_kappa_score, log_loss

c:\Users\zkdlx\miniconda3\envs\algorithm3.9\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\zkdlx\miniconda3\envs\algorithm3.9\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.11.0 and strictly below 2.14.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.10.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file

In [2]:
MODEL_NAME = "klue/bert-base"
model = TFBertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=3, from_pt=True)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# DATASET_URL = "https://raw.githubusercontent.com/ukairia777/finance_sentiment_corpus/main/finance_data.csv"
DATASET_NAME = "기사 긍부정 학습 데이터.csv"
# urllib.request.urlretrieve(DATASET_URL, 
#                            filename = DATASET_NAME
#                            )


In [4]:
data = pd.read_csv(DATASET_NAME)
data.head()

,Title,Contents,"label(긍정1, 부정0)"
0,태권도 유단자들의 살인 발차기…23살 청년을 죽였다[그해 오늘],[이데일리 한광범 기자] 2020년 1월 1일 새벽 3시 무렵. 새해를 맞아 여자친...,0
1,"다신 보고 싶지 않은 ‘이 사진’…세상에서 가장 괴로운 뉴스 [지구, 뭐래?]",[헤럴드경제 = 김상수 기자]정말 춥다. 조금만 기억을 되돌려보자. 겨울이 맞나 싶...,0
2,"'돈 많은 줄 알았는데'…래퍼 도끼, 세금 체납에 건보료도 밀려",래퍼 도끼(33)가 최근 3억 원 가량의 세금을 체납한 것으로 알려진 가운데 건강보...,0
3,"""생명 위협 느껴"" 히잡 벗은 이란 女 체스 선수, 보복 무서워 간 곳은",히잡을 벗고 국제 대회에 참가해 세계적 관심을 받았던 이란의 여성 체스선수 사라 카...,0
4,나체로 창고에 가둬' 지적장애 동생 학대한 친누나 부부 체포,말을 듣지 않는다는 이유로 지적장애인인 동생을 집 창고에 가두고 학대한 혐의로 20...,0


In [5]:
X_data = data['Contents']
y_data = data['label(긍정1, 부정0)']

In [6]:
TEST_SIZE = 0.2 # Train: Test = 8 :2 분리
RANDOM_STATE = 77
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                                    test_size = TEST_SIZE, 
                                                    random_state = RANDOM_STATE, 
                                                    stratify = y_data)

In [7]:
print(f"훈련 입력 데이터 개수: {len(X_train)}")
print(f"테스트 입력 데이터 개수: {len(X_test)}")

훈련 입력 데이터 개수: 79
테스트 입력 데이터 개수: 20


In [22]:
# 입력 데이터(문장) 길이 제한
MAX_SEQ_LEN = 512

In [23]:
def convert_data(X_data, y_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments, targets = [], [], [], []
    
    for X, y in tqdm(zip(X_data, y_data)):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        targets.append(y)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

In [24]:
def convert_data_x(X_data):
    # BERT 입력으로 들어가는 token, mask, segment, target 저장용 리스트
    tokens, masks, segments = [], [], []
    
    for X in tqdm(X_data):
        # token: 입력 문장 토큰화
        token = tokenizer.encode(X, truncation = True, padding = 'max_length', max_length = MAX_SEQ_LEN)
        
        # Mask: 토큰화한 문장 내 패딩이 아닌 경우 1, 패딩인 경우 0으로 초기화
        num_zeros = token.count(0)
        mask = [1] * (MAX_SEQ_LEN - num_zeros) + [0] * num_zeros
        
        # segment: 문장 전후관계 구분: 오직 한 문장이므로 모두 0으로 초기화
        segment = [0]*MAX_SEQ_LEN

        tokens.append(token)
        masks.append(mask)
        segments.append(segment)

    # numpy array로 저장
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)

    return [tokens, masks, segments]

In [25]:
# train 데이터를 Bert의 Input 타입에 맞게 변환
train_x, train_y = convert_data(X_train, y_train)
# test 데이터를 Bert의 Input 타입에 맞게 변환
test_x, test_y = convert_data(X_test, y_test)

79it [00:00, 177.32it/s]
20it [00:00, 230.73it/s]


In [26]:
# token, mask, segment 입력 정의
token_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_word_ids')
mask_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_masks')
segment_inputs = tf.keras.layers.Input((MAX_SEQ_LEN,), dtype = tf.int32, name = 'input_segment')
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])

In [27]:
bert_output = bert_outputs[0]

In [28]:
DROPOUT_RATE = 0.5
NUM_CLASS = 2
dropout = tf.keras.layers.Dropout(DROPOUT_RATE)(bert_output)
# binary-class classification 문제이므로 activation function은 softmax로 설정
sentiment_layer = tf.keras.layers.Dense(NUM_CLASS, activation='sigmoid', kernel_initializer = tf.keras.initializers.TruncatedNormal(stddev=0.02))(dropout)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_layer)

In [29]:
# 옵티마이저 Rectified Adam 하이퍼파리미터 조정
OPTIMIZER_NAME = 'RAdam'
LEARNING_RATE = 5e-5
TOTAL_STEPS = 10000
MIN_LR = 1e-5
WARMUP_PROPORTION = 0.1
EPSILON = 1e-8
CLIPNORM = 1.0
optimizer = tfa.optimizers.RectifiedAdam(learning_rate = LEARNING_RATE,
                                          total_steps = TOTAL_STEPS, 
                                          warmup_proportion = WARMUP_PROPORTION, 
                                          min_lr = MIN_LR, 
                                          epsilon = EPSILON,
                                          clipnorm = CLIPNORM)

In [30]:
# 감정분류 모델 컴파일
sentiment_model.compile(optimizer = optimizer, 
                        loss = tf.keras.losses.SparseCategoricalCrossentropy(), 
                        metrics = ['accuracy'])

In [31]:
MIN_DELTA = 1e-3
PATIENCE = 5

early_stopping = EarlyStopping(
    monitor = "val_accuracy", 
    min_delta = MIN_DELTA,
    patience = PATIENCE)

In [32]:
# 최고 성능의 모델 파일을 저장할 이름과 경로 설정
BEST_MODEL_NAME = './model/best_model.h5'
model_checkpoint = ModelCheckpoint(
    filepath = BEST_MODEL_NAME,
    monitor = "val_loss",
    mode = "min",
    save_best_only = True, # 성능 향상 시에만 모델 저장
    verbose = 1
)

In [33]:
callbacks = [early_stopping, model_checkpoint]

In [34]:
EPOCHS = 100
BATCH_SZIE = 8

sentiment_model.fit(train_x, train_y, 
                    epochs = EPOCHS, 
                    shuffle = True, 
                    batch_size = BATCH_SZIE, 
                    validation_data = (test_x, test_y),
                    callbacks = callbacks
                    )

Epoch 1/100
10/10 [==============================] - ETA: 0s - loss: 0.6930 - accuracy: 0.5823 
Epoch 1: val_loss improved from inf to 0.69293, saving model to ./model\best_model.h5
10/10 [==============================] - 220s 16s/step - loss: 0.6930 - accuracy: 0.5823 - val_loss: 0.6929 - val_accuracy: 0.4500
Epoch 2/100
10/10 [==============================] - ETA: 0s - loss: 0.6933 - accuracy: 0.4684 
Epoch 2: val_loss improved from 0.69293 to 0.69290, saving model to ./model\best_model.h5
10/10 [==============================] - 160s 16s/step - loss: 0.6933 - accuracy: 0.4684 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 3/100
 9/10 [==========================>...] - ETA: 16s - loss: 0.6932 - accuracy: 0.4722


KeyboardInterrupt



In [ ]:
# 최고 성능의 모델 불러오기
sentiment_model_best = tf.keras.models.load_model(BEST_MODEL_NAME,
                                                  custom_objects={'TFBertForSequenceClassification': TFBertForSequenceClassification})

In [ ]:
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

31/31 [==============================] - 52s 1s/step


In [ ]:
x = convert_data_x("미국 연수(캘리포니아 얼바인 거주) 시절 부러웠던 것 중 하나가 중증 장애인에 대한 배려였다. 아이 학교에 갈 때마다 장애인 학생들이 도우미의 도움을 받으며 밝은 표정으로 등하교하는 모습을 보고는 “이래서 장애인 부모들이 자녀를 미국에서 살게 하려고 애쓰는구나”하는 생각을 해보기도 했다.디지털 기술의 도움으로 우리나라에서도 이런 모습을 볼 수 있게 됐다. 에스케이텔레콤(SKT)·한국장애인고용공단·성남시(경기도)·모두의셔틀이 ‘장애인 이동권 및 고용복지 증진을 위한 공동협력 업무협약’을 맺고, 발달 장애를 가진 직장인들의 출퇴근을 돕는 ‘착한셔틀 모빌리티’(이하 착한셔틀) 시범서비스에 나선다. 장애인들에게 안전하고 편리한 이동권을 보장해 사회활동 참여 기회를 확대하고 장애인 일자리 생태계 관련 선순환 프로그램을 만들어 보자는 취지다. 이달 셋째 주부터 내년 3월까지 성남시서 시범서비스를 해본 뒤 전국으로 확대할 예정이다. 현재 성남시는 9곳의 직업재활시설에서 발달 장애인 70여명을 고용하고 있다. 시범서비스는 25인승 미니버스 9대로 이들의 출퇴근을 ‘집 대문부터 회사 현관까지(도어 투 도어)’ 지원한다. 버스마다 장애인 승하차 전문 보조원이 동승하고, 위성항법장치(GPS) 기반 위치확인 서비스 ‘스마트 지킴이’가 이용자의 현재 위치를 실시간으로 보호자에게 알려준다. 중증 발달 장애인들은 홀로 대중교통을 이용해 출퇴근하는 게 어려워, 가족이 동행하거나 장애인 콜택시 등을 이용해야 한다. 이 때문에 취업을 포기하는 경우가 많다. 지방자치단체별로 장애인 근로자들의 이동권 보장을 위해 교통비를 지원하거나 장애인 콜택시 바우처를 지급하는 등의 지원책을 펴고 있지만, 출퇴근 전용 셔틀 서비스 제공은 처음이다. 비용은 사업 참여자와 이용자들이 분담한다. 장애인고용공단은 손목시계 모양의 장애인 보조공학기기를 무상 제공하고, 성남시는 승하차 보조원 인건비를 댄다. 에스케이텔레콤은 장애인 실시간 위치확인 및 버스 위치·좌석 확인·예약 서비스를 제공한다. 이용자는 월 3만2380원(시범서비스 기간에는 2만1520원)의 이용료를 부담한다.장애인고용공단은 “착한셔틀을 통해 장애인 출퇴근 문제가 해결되면, 성남시에서만 500여명의 장애인이 추가 고용될 수 있다”고 분석했다. 에스케이텔레콤은 “시범서비스가 끝나고 본격 서비스로 전환할 때는 대상을 노약자, 임산부 등으로 확대하는 방안도 검토할 계획이다. 사업 확대로 이용자가 늘어나면 셔틀 운전기사와 장애인 승하차 보조원 등 서비스 운영을 위한 신규 고용 창출도 기대된다”고 설명했다.디지털 기술이 따뜻한 목적으로 사용되며 중증 장애인의 사회 참여를 돕고 고용 창출도 하는 모습이어서 반갑다.")

100%|██████████| 1314/1314 [00:00<00:00, 9717.43it/s]


In [121]:
pre = sentiment_model_best.predict(x)
label = np.argmax(pre, axis = 1)

42/42 [==============================] - 64s 1s/step


In [ ]:
predicted_value = sentiment_model_best.predict(test_x)
predicted_label = np.argmax(predicted_value, axis = 1)

In [ ]:
cl_report = classification_report(test_y, predicted_label, output_dict = True)
cl_report_df = pd.DataFrame(cl_report).transpose()
cl_report_df = cl_report_df.round(3)
print(cl_report_df)

In [ ]:
cf_matrix = confusion_matrix(test_y, predicted_label)
fig, ax = plt.subplots(figsize = (8,6))
sns.heatmap(cf_matrix, annot = True, fmt = 'd')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()